## Лабораторна робота **No. 2.1** студентки **Іванченко О. В.**

Тема: Нейронні мережі для задач класифікації.

---
###Завдання  1. Вибір варіанту завдання
  - Оберіть один з 10 варіантів завдань відповідно до свого номеру в групі або за вказівкою викладача.

        Варіант 5: Wine Dataset
        Опис: Класифікація зразків вина за різними сортами на основі хімічного аналізу.
              Особливості набору даних: 13 ознак, 3 класи (різні типи вина)

---
###Завдання  2. Підготовка даних
  - Імпортуйте необхідні бібліотеки
  - Завантажте дані відповідно до вашого варіанту
  - Візуалізуйте дані для розуміння їх структури
  - Поділіть дані на тренувальну та тестову вибірки
  - Виконайте нормалізацію/стандартизацію даних

In [1]:
# встановлюємо пакет, приховуємо вивід
!pip install gradio > /dev/null 2>&1

In [2]:
# Імпортування бібліотек

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.datasets import load_breast_cancer, load_wine, load_digits, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, auc
import gradio as gr
import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Функції для завантаження наборів даних
def load_data(variant):
    if variant == 1:  # Breast Cancer Wisconsin
        data = load_breast_cancer()
        X = data.data
        y = data.target
        feature_names = data.feature_names
        class_names = data.target_names
        return X, y, feature_names, class_names, "Breast Cancer Wisconsin"

    elif variant == 10:  # MNIST Digits
        digits = load_digits()
        X = digits.data  # 64 пікселі (8x8)
        y = digits.target
        feature_names = [f"pixel_{i}" for i in range(X.shape[1])]
        class_names = [str(i) for i in range(10)]
        return X, y, feature_names, class_names, "MNIST Digits"

    elif variant == 3:  # Pima Indians Diabetes
        try:
            url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
            columns = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]
            data = pd.read_csv(url, names=columns)
            X = data.iloc[:, :-1].values
            y = data.iloc[:, -1].values
            feature_names = columns[:-1]
            class_names = ["No Diabetes", "Diabetes"]
            return X, y, feature_names, class_names, "Pima Indians Diabetes"
        except:
            # Fallback на випадок, якщо набір даних недоступний
            data = load_breast_cancer()
            X = data.data
            y = data.target
            feature_names = data.feature_names
            class_names = data.target_names
            return X, y, feature_names, class_names, "Fallback: Breast Cancer Wisconsin (URL Error)"

    elif variant == 4:  # Wine Quality (Red)
        try:
            url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
            data = pd.read_csv(url, sep=';')
            X = data.drop('quality', axis=1).values
            y = (data['quality'] >= 6).astype(int)  # Бінаризація: 0 (низька) і 1 (висока)
            feature_names = data.drop('quality', axis=1).columns.tolist()
            class_names = ["Low Quality", "High Quality"]
            print(f"Unique values in y: {np.unique(y)}")  # Має вивести [0, 1]
            return X, y, feature_names, class_names, "Wine Quality (Red)"
        except:
            # Fallback на випадок, якщо набір даних недоступний
            data = load_breast_cancer()
            X = data.data
            y = data.target
            feature_names = data.feature_names
            class_names = data.target_names
            return X, y, feature_names, class_names, "Fallback: Breast Cancer Wisconsin (URL Error)"

    elif variant == 5:  # Wine Dataset
        data = load_wine()
        X = data.data
        y = data.target
        feature_names = data.feature_names
        class_names = data.target_names
        return X, y, feature_names, class_names, "Wine Dataset"

    elif variant == 6:  # Titanic
        try:
            url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
            data = pd.read_csv(url)
            # Базова обробка даних
            data = data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
            # Заповнення пропущених значень
            data['Age'].fillna(data['Age'].median(), inplace=True)
            data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

            # Перетворення категоріальних змінних
            data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
            data['Embarked'] = data['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

            X = data.drop('Survived', axis=1).values
            y = data['Survived'].values
            feature_names = data.drop('Survived', axis=1).columns.tolist()
            class_names = ["Did not survive", "Survived"]
            return X, y, feature_names, class_names, "Titanic"
        except:
            # Fallback на випадок, якщо набір даних недоступний
            data = load_breast_cancer()
            X = data.data
            y = data.target
            feature_names = data.feature_names
            class_names = data.target_names
            return X, y, feature_names, class_names, "Fallback: Breast Cancer Wisconsin (URL Error)"

    elif variant == 7:  # Heart Disease
        try:
            url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
            columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
            data = pd.read_csv(url, names=columns, na_values='?')
            # Обробка пропущених значень
            data.dropna(inplace=True)

            # Перетворення цільової змінної на бінарну
            data['target'] = (data['target'] > 0).astype(int)

            X = data.drop('target', axis=1).values
            y = data['target'].values
            feature_names = columns[:-1]
            class_names = ["No Heart Disease", "Heart Disease"]
            return X, y, feature_names, class_names, "Heart Disease"
        except:
            # Fallback на випадок, якщо набір даних недоступний
            data = load_breast_cancer()
            X = data.data
            y = data.target
            feature_names = data.feature_names
            class_names = data.target_names
            return X, y, feature_names, class_names, "Fallback: Breast Cancer Wisconsin (URL Error)"

    elif variant == 8:  # Iris
        try:
            url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
            columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
            data = pd.read_csv(url, names=columns)

            # Перетворення категоріальних міток класу
            class_mapping = {
                'Iris-setosa': 0,
                'Iris-versicolor': 1,
                'Iris-virginica': 2
            }
            data['class'] = data['class'].map(class_mapping)

            X = data.iloc[:, :-1].values
            y = data.iloc[:, -1].values
            feature_names = columns[:-1]
            class_names = ["Setosa", "Versicolor", "Virginica"]
            return X, y, feature_names, class_names, "Iris"
        except:
            # Fallback на випадок, якщо набір даних недоступний
            data = load_breast_cancer()
            X = data.data
            y = data.target
            feature_names = data.feature_names
            class_names = data.target_names
            return X, y, feature_names, class_names, "Fallback: Breast Cancer Wisconsin (URL Error)"

    elif variant == 9:  # Mushroom Dataset
        try:
            url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
            columns = ['class', 'cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor',
                    'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color',
                    'stalk_shape', 'stalk_root', 'stalk_surface_above_ring',
                    'stalk_surface_below_ring', 'stalk_color_above_ring',
                    'stalk_color_below_ring', 'veil_type', 'veil_color',
                    'ring_number', 'ring_type', 'spore_print_color', 'population', 'habitat']

            data = pd.read_csv(url, names=columns)

            # Замінюємо категоріальні ознаки на числові
            data_encoded = pd.get_dummies(data.drop('class', axis=1))

            # Перетворюємо цільову змінну
            data['class'] = data['class'].map({'e': 0, 'p': 1})

            X = data_encoded.values
            y = data['class'].values
            feature_names = data_encoded.columns.tolist()
            class_names = ["Edible", "Poisonous"]
            return X, y, feature_names, class_names, "Mushroom Dataset"
        except:
            # Fallback на випадок, якщо набір даних недоступний
            data = load_breast_cancer()
            X = data.data
            y = data.target
            feature_names = data.feature_names
            class_names = data.target_names
            return X, y, feature_names, class_names, "Fallback: Breast Cancer Wisconsin (URL Error)"

    elif variant == 2:  # Fashion MNIST (обмежений набір)
        try:
            # Завантажуємо обмежений набір Fashion MNIST для швидкості
            (X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

            # Беремо лише підмножину для швидкості
            subset_size = 5000
            X = np.vstack((X_train[:subset_size], X_test[:subset_size]))
            y = np.concatenate((y_train[:subset_size], y_test[:subset_size]))

            # Нормалізація і реформування даних
            X = X.reshape(-1, 784) / 255.0

            feature_names = [f"pixel_{i}" for i in range(X.shape[1])]
            class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                           "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
            return X, y, feature_names, class_names, "Fashion MNIST (subset)"
        except:
            # Fallback на випадок, якщо набір даних недоступний
            data = load_breast_cancer()
            X = data.data
            y = data.target
            feature_names = data.feature_names
            class_names = data.target_names
            return X, y, feature_names, class_names, "Fallback: Breast Cancer Wisconsin (URL Error)"

    else:
        # За замовчуванням - Breast Cancer Wisconsin
        data = load_breast_cancer()
        X = data.data
        y = data.target
        feature_names = data.feature_names
        class_names = data.target_names
        return X, y, feature_names, class_names, "Breast Cancer Wisconsin (Default)"

In [4]:
# Візуалізація сирих даних
"""
from sklearn.decomposition import PCA

# PCA до 2 компонентів
pca = PCA(n_components=2)
X2 = pca.fit_transform(X)

x1 = X2[:,:1]
y1 = X2[:,1:]

# Побудова графіка
plt.figure(figsize=(8, 6))
plt.scatter(x1,y1)
plt.show()

X2.shape
"""

'\nfrom sklearn.decomposition import PCA\n\n# PCA до 2 компонентів\npca = PCA(n_components=2)\nX2 = pca.fit_transform(X)\n\nx1 = X2[:,:1]\ny1 = X2[:,1:]\n\n# Побудова графіка\nplt.figure(figsize=(8, 6))\nplt.scatter(x1,y1)\nplt.show()\n\nX2.shape\n'

In [5]:
# Поділ даних на тренувальну та тестову вибірки
"""
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
"""

'\nX_train, X_test, y_train, y_test = train_test_split(\n    X, y, test_size=0.2, random_state=42, stratify=y\n)\n'

In [6]:
# Стандартизаці"""я даних
"""
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
"""

'\nscaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)  \nX_test = scaler.fit_transform(X_test)      \n'

In [7]:
# Створення фігури з двома subplot'ами
"""
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# PCA для тренувальних даних
pca = PCA(n_components=2)
X2_train = pca.fit_transform(X_train)
axs[0].scatter(X2_train[:, 0], X2_train[:, 1])
axs[0].set_title("PCA на X_train")

# PCA для тестових даних
pca = PCA(n_components=2)
X2_test = pca.fit_transform(X_test)
axs[1].scatter(X2_test[:, 0], X2_test[:, 1])
axs[1].set_title("PCA на X_test")

plt.tight_layout()
plt.show()
"""

'\nfig, axs = plt.subplots(1, 2, figsize=(14, 6))\n\n# PCA для тренувальних даних\npca = PCA(n_components=2)\nX2_train = pca.fit_transform(X_train)\naxs[0].scatter(X2_train[:, 0], X2_train[:, 1])\naxs[0].set_title("PCA на X_train")\n\n# PCA для тестових даних\npca = PCA(n_components=2)\nX2_test = pca.fit_transform(X_test)\naxs[1].scatter(X2_test[:, 0], X2_test[:, 1])\naxs[1].set_title("PCA на X_test")\n\nplt.tight_layout()\nplt.show()\n'

---
###Завдання 3. Реалізація моделей нейронних мереж
  - Створіть простий багатошаровий перцептрон з використанням TensorFlow/Keras
  - Реалізуйте альтернативну модель з використанням scikit-learn
  - Експериментуйте з різними архітектурами (кількістю шарів, нейронів, функцій активації)

In [8]:
# Функція для побудови TensorFlow/Keras моделі

def build_keras_model(input_dim, num_classes, architecture, activation='relu', hidden_units=(32, 16), dropout_rate=0.2):
    if architecture == "simple":
        model = Sequential([
            Dense(hidden_units[0], activation=activation, input_shape=(input_dim,)),
            Dense(hidden_units[1], activation=activation),
            Dense(1 if num_classes == 2 else num_classes,
                  activation='sigmoid' if num_classes == 2 else 'softmax')
        ])
    elif architecture == "cnn":
        if input_dim == 784:  # Для Fashion MNIST
            input_img = Input(shape=(input_dim,))
            x = tf.reshape(input_img, [-1, 28, 28, 1])
            x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
            x = MaxPooling2D(pool_size=(2, 2))(x)
            x = Flatten()(x)
            x = Dense(128, activation='relu')(x)
            x = Dropout(0.5)(x)
            outputs = Dense(1 if num_classes == 2 else num_classes,
                            activation='sigmoid' if num_classes == 2 else 'softmax')(x)
            model = Model(inputs=input_img, outputs=outputs)
        elif input_dim == 64:  # Для MNIST Digits
            input_img = Input(shape=(input_dim,))
            x = tf.reshape(input_img, [-1, 8, 8, 1])  # Правильний розмір для 8x8
            x = Conv2D(32, kernel_size=(2, 2), activation='relu')(x)  # Менше ядро
            x = MaxPooling2D(pool_size=(2, 2))(x)
            x = Flatten()(x)
            x = Dense(128, activation='relu')(x)
            x = Dropout(0.5)(x)
            outputs = Dense(1 if num_classes == 2 else num_classes,
                            activation='sigmoid' if num_classes == 2 else 'softmax')(x)
            model = Model(inputs=input_img, outputs=outputs)
        else:
            # Fallback на повнозв’язну модель
            model = Sequential([
                Dense(128, activation='relu', input_shape=(input_dim,)),
                Dense(64, activation='relu'),
                Dense(1 if num_classes == 2 else num_classes,
                      activation='sigmoid' if num_classes == 2 else 'softmax')
            ])
    else:
        # Fallback на просту модель
        model = Sequential([
            Dense(hidden_units[0], activation=activation, input_shape=(input_dim,)),
            Dense(hidden_units[1], activation=activation),
            Dense(1 if num_classes == 2 else num_classes,
                  activation='sigmoid' if num_classes == 2 else 'softmax')
        ])

    return model


In [9]:
# Функція для тренування TensorFlow/Keras моделі
def train_keras_model(model, X_train, y_train, X_test, y_test, num_classes,
                      optimizer='adam', learning_rate=0.001, batch_size=32, epochs=50,
                      patience=5, verbose=0):

    # Перетворення міток тільки для багатокласової класифікації
    if num_classes > 2:
        y_train_cat = to_categorical(y_train, num_classes)
        y_test_cat = to_categorical(y_test, num_classes)
    else:
        y_train_cat = y_train  # Залишаємо як є для двокласової
        y_test_cat = y_test

    # Налаштування оптимізатора
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    else:
        opt = RMSprop(learning_rate=learning_rate)

    # Компіляція моделі
    model.compile(
        optimizer=opt,
        loss='binary_crossentropy' if num_classes == 2 else 'categorical_crossentropy',
        metrics=['accuracy']
    )

    # Раннє зупинення навчання
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True
    )

    # Тренування моделі
    start_time = time.time()
    history = model.fit(
        X_train, y_train_cat,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.1,
        callbacks=[early_stopping],
        verbose=verbose
    )
    training_time = time.time() - start_time

    # Оцінка моделі
    test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)

    # Передбачення
    if num_classes > 2:
        y_pred_proba = model.predict(X_test)
        y_pred = np.argmax(y_pred_proba, axis=1)
    else:
        y_pred_proba = model.predict(X_test).flatten()
        y_pred = (y_pred_proba > 0.5).astype(int)

    return {
        'model': model,
        'history': history,
        'training_time': training_time,
        'test_accuracy': test_acc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

In [10]:
# Функція для тренування scikit-learn моделі

def train_sklearn_model(X_train, y_train, X_test, y_test, model_type='mlp',
                       hidden_layer_sizes=(32, 16), alpha=0.0001, max_iter=200):

    start_time = time.time()

    if model_type == 'mlp':
        model = MLPClassifier(
            hidden_layer_sizes=hidden_layer_sizes,
            activation='relu',
            solver='adam',
            alpha=alpha,
            max_iter=max_iter,
            random_state=42
        )
    elif model_type == 'logistic':
        model = LogisticRegression(
            C=1/alpha if alpha > 0 else 1e6,
            max_iter=max_iter,
            random_state=42
        )
    else:
        # За замовчуванням - MLPClassifier
        model = MLPClassifier(
            hidden_layer_sizes=hidden_layer_sizes,
            activation='relu',
            solver='adam',
            alpha=alpha,
            max_iter=max_iter,
            random_state=42
        )

    # Навчання моделі
    model.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Передбачення
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    # Оцінка моделі
    accuracy = accuracy_score(y_test, y_pred)

    return {
        'model': model,
        'training_time': training_time,
        'test_accuracy': accuracy,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

In [11]:
# Функція для оцінки та порівняння моделей
def evaluate_models(y_test, keras_results, sklearn_results, class_names):
    num_classes = len(class_names)

    # Для бінарної класифікації
    if num_classes == 2:
        keras_precision = precision_score(y_test, keras_results['y_pred'])
        keras_recall = recall_score(y_test, keras_results['y_pred'])
        keras_f1 = f1_score(y_test, keras_results['y_pred'])

        sklearn_precision = precision_score(y_test, sklearn_results['y_pred'])
        sklearn_recall = recall_score(y_test, sklearn_results['y_pred'])
        sklearn_f1 = f1_score(y_test, sklearn_results['y_pred'])

        # ROC криві
        keras_fpr, keras_tpr, _ = roc_curve(y_test, keras_results['y_pred_proba'] if num_classes == 2 else keras_results['y_pred_proba'][:, 1])
        keras_auc = auc(keras_fpr, keras_tpr)

        sklearn_fpr, sklearn_tpr, _ = roc_curve(y_test, sklearn_results['y_pred_proba'][:, 1])
        sklearn_auc = auc(sklearn_fpr, sklearn_tpr)

        roc_data = {
            'keras': {'fpr': keras_fpr, 'tpr': keras_tpr, 'auc': keras_auc},
            'sklearn': {'fpr': sklearn_fpr, 'tpr': sklearn_tpr, 'auc': sklearn_auc}
        }
    else:
        # Для багатокласової класифікації
        keras_precision = precision_score(y_test, keras_results['y_pred'], average='weighted')
        keras_recall = recall_score(y_test, keras_results['y_pred'], average='weighted')
        keras_f1 = f1_score(y_test, keras_results['y_pred'], average='weighted')

        sklearn_precision = precision_score(y_test, sklearn_results['y_pred'], average='weighted')
        sklearn_recall = recall_score(y_test, sklearn_results['y_pred'], average='weighted')
        sklearn_f1 = f1_score(y_test, sklearn_results['y_pred'], average='weighted')

        roc_data = None  # ROC криві не обчислюються для багатокласової класифікації в цьому прикладі

    # Матриці помилок
    keras_cm = confusion_matrix(y_test, keras_results['y_pred'])
    sklearn_cm = confusion_matrix(y_test, sklearn_results['y_pred'])

    return {
        'keras': {
            'accuracy': keras_results['test_accuracy'],
            'precision': keras_precision,
            'recall': keras_recall,
            'f1': keras_f1,
            'cm': keras_cm,
            'training_time': keras_results['training_time']
        },
        'sklearn': {
            'accuracy': sklearn_results['test_accuracy'],
            'precision': sklearn_precision,
            'recall': sklearn_recall,
            'f1': sklearn_f1,
            'cm': sklearn_cm,
            'training_time': sklearn_results['training_time']
        },
        'roc_data': roc_data
    }


 ---
###Завдання 4. Навчання та оцінка моделей
  - Визначте гіперпараметри (швидкість навчання, розмір батчу, кількість епох)
  - Навчіть моделі на тренувальних даних
  - Оцініть моделі на тестових даних
  - Порівняйте точність, час навчання та інші метрики

In [12]:
# Функція для відображення метрик моделей
def plot_model_metrics(evaluation_results):
    keras_metrics = evaluation_results['keras']
    sklearn_metrics = evaluation_results['sklearn']

    metrics = ['accuracy', 'precision', 'recall', 'f1']
    keras_values = [keras_metrics[m] for m in metrics]
    sklearn_values = [sklearn_metrics[m] for m in metrics]

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Порівняння метрик
    x = range(len(metrics))
    bar_width = 0.35

    axes[0].bar([i - bar_width/2 for i in x], keras_values, bar_width, label='Keras', color='#3498db')
    axes[0].bar([i + bar_width/2 for i in x], sklearn_values, bar_width, label='Scikit-learn', color='#e74c3c')

    axes[0].set_xticks(x)
    axes[0].set_xticklabels(metrics)
    axes[0].set_ylabel('Score')
    axes[0].set_title('Model Performance Metrics')
    axes[0].legend()
    axes[0].set_ylim(0, 1)

    # Порівняння часу тренування
    times = [keras_metrics['training_time'], sklearn_metrics['training_time']]
    axes[1].bar(['Keras', 'Scikit-learn'], times, color=['#3498db', '#e74c3c'])
    axes[1].set_ylabel('Time (seconds)')
    axes[1].set_title('Training Time Comparison')

    plt.tight_layout()
    return fig

In [13]:

# Функція для візуалізації межі прийняття рішень (для 2D даних)
def plot_decision_boundary(X, y, keras_model, sklearn_model, num_classes):
    # Працює тільки з 2 ознаками
    if X.shape[1] != 2:
        return None

    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Створюємо сітку точок
    h = 0.02  # крок сітки
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # Передбачення для Keras моделі
    if num_classes > 2:
        Z_keras = np.argmax(keras_model.predict(np.c_[xx.ravel(), yy.ravel()]), axis=1)
    else:
        Z_keras = (keras_model.predict(np.c_[xx.ravel(), yy.ravel()]) > 0.5).astype(int).flatten()
    Z_keras = Z_keras.reshape(xx.shape)

    # Передбачення для Scikit-learn моделі
    Z_sklearn = sklearn_model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z_sklearn = Z_sklearn.reshape(xx.shape)

    # Відображення меж прийняття рішень
    for ax, Z, title in zip(axes, [Z_keras, Z_sklearn], ['Keras Model', 'Scikit-learn Model']):
        ax.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k')
        ax.set_title(title)
        ax.set_xlabel('Feature 1')
        ax.set_ylabel('Feature 2')

    plt.tight_layout()
    return fig


In [14]:
# Основна функція для навчання та оцінки моделей
def train_and_evaluate_models(variant, test_size, keras_architecture, sklearn_model_type,
                            hidden_units, activation, learning_rate, optimizer, batch_size,
                            epochs, dropout_rate, alpha, max_iter, random_state):
    # Завантаження даних
    X, y, feature_names, class_names, dataset_name = load_data(variant)
    num_classes = len(class_names)

    # Розділення на тренувальні та тестові дані
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size/100, random_state=random_state)

    # Стандартизація даних
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Побудова та навчання Keras моделі
    keras_model = build_keras_model(
        input_dim=X_train_scaled.shape[1],
        num_classes=num_classes,
        architecture=keras_architecture,
        activation=activation,
        hidden_units=hidden_units,
        dropout_rate=dropout_rate
    )

    keras_results = train_keras_model(
        model=keras_model,
        X_train=X_train_scaled,
        y_train=y_train,
        X_test=X_test_scaled,
        y_test=y_test,
        num_classes=num_classes,
        optimizer=optimizer,
        learning_rate=learning_rate,
        batch_size=batch_size,
        epochs=epochs
    )

    # Навчання Scikit-learn моделі
    sklearn_results = train_sklearn_model(
        X_train=X_train_scaled,
        y_train=y_train,
        X_test=X_test_scaled,
        y_test=y_test,
        model_type=sklearn_model_type,
        hidden_layer_sizes=hidden_units,
        alpha=alpha,
        max_iter=max_iter
    )

    # Оцінка моделей
    evaluation_results = evaluate_models(
        y_test=y_test,
        keras_results=keras_results,
        sklearn_results=sklearn_results,
        class_names=class_names
    )

    # Підготовка результатів
    results = {
        'dataset_name': dataset_name,
        'class_names': class_names,
        'num_classes': num_classes,
        'feature_names': feature_names,
        'X_test': X_test_scaled,
        'y_test': y_test,
        'keras_model': keras_model,
        'sklearn_model': sklearn_results['model'],
        'keras_history': keras_results['history'],
        'keras_pred': keras_results['y_pred'],
        'sklearn_pred': sklearn_results['y_pred'],
        'evaluation_results': evaluation_results
    }

    return results


In [15]:
# Функція для оцінки та порівняння моделей
def evaluate_models(y_test, keras_results, sklearn_results, class_names):
    num_classes = len(class_names)

    # Для бінарної класифікації
    if num_classes == 2:
        keras_precision = precision_score(y_test, keras_results['y_pred'])
        keras_recall = recall_score(y_test, keras_results['y_pred'])
        keras_f1 = f1_score(y_test, keras_results['y_pred'])

        sklearn_precision = precision_score(y_test, sklearn_results['y_pred'])
        sklearn_recall = recall_score(y_test, sklearn_results['y_pred'])
        sklearn_f1 = f1_score(y_test, sklearn_results['y_pred'])

        # ROC криві
        keras_fpr, keras_tpr, _ = roc_curve(y_test, keras_results['y_pred_proba'] if num_classes == 2 else keras_results['y_pred_proba'][:, 1])
        keras_auc = auc(keras_fpr, keras_tpr)

        sklearn_fpr, sklearn_tpr, _ = roc_curve(y_test, sklearn_results['y_pred_proba'][:, 1])
        sklearn_auc = auc(sklearn_fpr, sklearn_tpr)

        roc_data = {
            'keras': {'fpr': keras_fpr, 'tpr': keras_tpr, 'auc': keras_auc},
            'sklearn': {'fpr': sklearn_fpr, 'tpr': sklearn_tpr, 'auc': sklearn_auc}
        }
    else:
        # Для багатокласової класифікації
        keras_precision = precision_score(y_test, keras_results['y_pred'], average='weighted')
        keras_recall = recall_score(y_test, keras_results['y_pred'], average='weighted')
        keras_f1 = f1_score(y_test, keras_results['y_pred'], average='weighted')

        sklearn_precision = precision_score(y_test, sklearn_results['y_pred'], average='weighted')
        sklearn_recall = recall_score(y_test, sklearn_results['y_pred'], average='weighted')
        sklearn_f1 = f1_score(y_test, sklearn_results['y_pred'], average='weighted')

        roc_data = None  # ROC криві не обчислюються для багатокласової класифікації в цьому прикладі

    # Матриці помилок
    keras_cm = confusion_matrix(y_test, keras_results['y_pred'])
    sklearn_cm = confusion_matrix(y_test, sklearn_results['y_pred'])

    return {
        'keras': {
            'accuracy': keras_results['test_accuracy'],
            'precision': keras_precision,
            'recall': keras_recall,
            'f1': keras_f1,
            'cm': keras_cm,
            'training_time': keras_results['training_time']
        },
        'sklearn': {
            'accuracy': sklearn_results['test_accuracy'],
            'precision': sklearn_precision,
            'recall': sklearn_recall,
            'f1': sklearn_f1,
            'cm': sklearn_cm,
            'training_time': sklearn_results['training_time']
        },
        'roc_data': roc_data
    }


 ---
###Завдання 5. Аналіз результатів
- Візуалізуйте криві навчання
- Побудуйте матриці помилок
- Для бінарної класифікації побудуйте ROC криві
- Проаналізуйте приклади помилково класифікованих зразків
- Спробуйте різні методи для покращення продуктивності

In [16]:
# Функція для відображення кривої навчання

def plot_keras_learning_curves(history):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Графік втрат
    axes[0].plot(history.history['loss'], label='Train Loss')
    axes[0].plot(history.history['val_loss'], label='Validation Loss')
    axes[0].set_title('Loss Curves')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()

    # Графік точності
    axes[1].plot(history.history['accuracy'], label='Train Accuracy')
    axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[1].set_title('Accuracy Curves')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()

    plt.tight_layout()
    return fig

In [17]:
# Функція для відображення матриць помилок

def plot_confusion_matrices(evaluation_results, class_names):
    keras_cm = evaluation_results['keras']['cm']
    sklearn_cm = evaluation_results['sklearn']['cm']

    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Матриця помилок для Keras моделі
    sns.heatmap(keras_cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
                xticklabels=class_names, yticklabels=class_names)
    axes[0].set_title('Keras Model Confusion Matrix')
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('True')

    # Матриця помилок для Scikit-learn моделі
    sns.heatmap(sklearn_cm, annot=True, fmt='d', cmap='Reds', ax=axes[1],
                xticklabels=class_names, yticklabels=class_names)
    axes[1].set_title('Scikit-learn Model Confusion Matrix')
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('True')

    plt.tight_layout()
    return fig

In [18]:
# Функція для відображення ROC кривих (для бінарної класифікації)

def plot_roc_curves(evaluation_results):
    roc_data = evaluation_results['roc_data']

    if roc_data is None:
        return None

    fig, ax = plt.subplots(figsize=(10, 6))

    # ROC крива для Keras моделі
    ax.plot(roc_data['keras']['fpr'], roc_data['keras']['tpr'],
            label=f'Keras (AUC = {roc_data["keras"]["auc"]:.3f})',
            color='#3498db', linewidth=2)

    # ROC крива для Scikit-learn моделі
    ax.plot(roc_data['sklearn']['fpr'], roc_data['sklearn']['tpr'],
            label=f'Scikit-learn (AUC = {roc_data["sklearn"]["auc"]:.3f})',
            color='#e74c3c', linewidth=2)

    # Базова лінія
    ax.plot([0, 1], [0, 1], 'k--', label='Random Guess')

    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver Operating Characteristic (ROC) Curves')
    ax.legend(loc='lower right')

    return fig


In [19]:
# Функція для відображення прикладів помилково класифікованих зразків
def plot_misclassifications(X_test, y_test, keras_pred, sklearn_pred, dataset_name, class_names):
    # Визначення, які зразки були неправильно класифіковані
    keras_errors = (y_test != keras_pred)
    sklearn_errors = (y_test != sklearn_pred)

    # Зразки, які обидві моделі класифікували неправильно
    common_errors = keras_errors & sklearn_errors

    # Зразки, які Keras класифікував правильно, а Scikit-learn - ні
    keras_correct_sklearn_wrong = ~keras_errors & sklearn_errors

    # Зразки, які Scikit-learn класифікував правильно, а Keras - ні
    keras_wrong_sklearn_correct = keras_errors & ~sklearn_errors

    # Перевірка, чи це дані зображень
    is_image_data = dataset_name in ["MNIST Digits", "Fashion MNIST (subset)"]

    fig = plt.figure(figsize=(15, 10))
    plt.suptitle(f"Misclassification Examples", fontsize=16)

    # Визначення зразків для відображення
    if np.sum(common_errors) > 0:
        indices = np.where(common_errors)[0]
        n_samples = min(5, len(indices))
        selected_indices = indices[:n_samples]

        for i, idx in enumerate(selected_indices):
            plt.subplot(3, 5, i + 1)

            if is_image_data:
                # Якщо це зображення, відображаємо його як зображення
                img = X_test[idx].reshape(28, 28)
                plt.imshow(img, cmap='gray')
            else:
                # Для інших даних відображаємо гістограму ознак
                plt.bar(range(min(10, X_test.shape[1])), X_test[idx][:min(10, X_test.shape[1])])

            plt.title(f"True: {class_names[y_test[idx]]}\nBoth Wrong", fontsize=9)
            plt.axis('off')

    # Зразки, які Keras класифікував правильно, а Scikit-learn - ні
    if np.sum(keras_correct_sklearn_wrong) > 0:
        indices = np.where(keras_correct_sklearn_wrong)[0]
        n_samples = min(5, len(indices))
        selected_indices = indices[:n_samples]

        for i, idx in enumerate(selected_indices):
            plt.subplot(3, 5, i + 6)

            if is_image_data:
                img = X_test[idx].reshape(28, 28)
                plt.imshow(img, cmap='gray')
            else:
                plt.bar(range(min(10, X_test.shape[1])), X_test[idx][:min(10, X_test.shape[1])])

            plt.title(f"True: {class_names[y_test[idx]]}\nKeras Correct", fontsize=9)
            plt.axis('off')

    # Зразки, які Scikit-learn класифікував правильно, а Keras - ні
    if np.sum(keras_wrong_sklearn_correct) > 0:
        indices = np.where(keras_wrong_sklearn_correct)[0]
        n_samples = min(5, len(indices))
        selected_indices = indices[:n_samples]

        for i, idx in enumerate(selected_indices):
            plt.subplot(3, 5, i + 11)

            if is_image_data:
                img = X_test[idx].reshape(28, 28)
                plt.imshow(img, cmap='gray')
            else:
                plt.bar(range(min(10, X_test.shape[1])), X_test[idx][:min(10, X_test.shape[1])])

            plt.title(f"True: {class_names[y_test[idx]]}\nScikit-learn Correct", fontsize=9)
            plt.axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    return fig

 ---
###Завдання 6. Створення інтерактивного інтерфейсу
- Використайте gradio для створення інтерактивного інтерфейсу
- Додайте можливість вибору різних параметрів моделі
- Забезпечте візуалізацію результатів

In [20]:
# Функція для представлення результатів у вигляді текстового звіту
def generate_results_text(results):
    dataset_name = results['dataset_name']
    num_classes = results['num_classes']
    keras_metrics = results['evaluation_results']['keras']
    sklearn_metrics = results['evaluation_results']['sklearn']

    output_text = f"## Результати аналізу для набору даних: {dataset_name}\n\n"
    output_text += f"Кількість класів: {num_classes}\n\n"

    output_text += "### Метрики Keras моделі:\n"
    output_text += f"- Точність (Accuracy): {keras_metrics['accuracy']:.4f}\n"
    output_text += f"- Precision: {keras_metrics['precision']:.4f}\n"
    output_text += f"- Recall: {keras_metrics['recall']:.4f}\n"
    output_text += f"- F1-міра: {keras_metrics['f1']:.4f}\n"
    output_text += f"- Час навчання: {keras_metrics['training_time']:.2f} секунд\n\n"

    output_text += "### Метрики Scikit-learn моделі:\n"
    output_text += f"- Точність (Accuracy): {sklearn_metrics['accuracy']:.4f}\n"
    output_text += f"- Precision: {sklearn_metrics['precision']:.4f}\n"
    output_text += f"- Recall: {sklearn_metrics['recall']:.4f}\n"
    output_text += f"- F1-міра: {sklearn_metrics['f1']:.4f}\n"
    output_text += f"- Час навчання: {sklearn_metrics['training_time']:.2f} секунд\n\n"

    # Додаємо порівняння
    keras_correct = np.sum(results['y_test'] == results['keras_pred'])
    sklearn_correct = np.sum(results['y_test'] == results['sklearn_pred'])
    total_samples = len(results['y_test'])

    output_text += "### Порівняння моделей:\n"
    output_text += f"- Загальна кількість зразків у тестовому наборі: {total_samples}\n"
    output_text += f"- Keras правильно класифікувала: {keras_correct} зразків ({keras_correct/total_samples*100:.2f}%)\n"
    output_text += f"- Scikit-learn правильно класифікувала: {sklearn_correct} зразків ({sklearn_correct/total_samples*100:.2f}%)\n"

    # Зразки, які обидві моделі класифікували неправильно
    both_wrong = np.sum((results['y_test'] != results['keras_pred']) & (results['y_test'] != results['sklearn_pred']))
    output_text += f"- Кількість зразків, неправильно класифікованих обома моделями: {both_wrong} ({both_wrong/total_samples*100:.2f}%)\n"

    return output_text

In [21]:
def gradio_interface(variant, test_size, keras_architecture, sklearn_model_type,
                   first_layer_units, second_layer_units, activation, learning_rate,
                   optimizer, batch_size, epochs, dropout_rate, alpha, max_iter, random_state):

    hidden_units = (first_layer_units, second_layer_units)

    try:
        results = train_and_evaluate_models(
            variant=variant,
            test_size=test_size,
            keras_architecture=keras_architecture,
            sklearn_model_type=sklearn_model_type,
            hidden_units=hidden_units,
            activation=activation,
            learning_rate=learning_rate,
            optimizer=optimizer,
            batch_size=batch_size,
            epochs=epochs,
            dropout_rate=dropout_rate,
            alpha=alpha,
            max_iter=max_iter,
            random_state=random_state
        )

        # Генерація текстового звіту
        output_text = generate_results_text(results)

        # Візуалізація
        learning_curve_plot = plot_keras_learning_curves(results['keras_history'])
        metrics_plot = plot_model_metrics(results['evaluation_results'])
        cm_plot = plot_confusion_matrices(results['evaluation_results'],
                                       results['class_names'][:min(10, len(results['class_names']))])

        misclassifications_plot = plot_misclassifications(
            results['X_test'], results['y_test'],
            results['keras_pred'], results['sklearn_pred'],
            results['dataset_name'], results['class_names']
        )

        # ROC криві для бінарної класифікації
        if results['num_classes'] == 2:
            roc_plot = plot_roc_curves(results['evaluation_results'])
        else:
            roc_plot = None

        # Межа прийняття рішень для 2D даних
        if results['X_test'].shape[1] == 2:
            decision_boundary_plot = plot_decision_boundary(
                results['X_test'], results['y_test'],
                results['keras_model'], results['sklearn_model'],
                results['num_classes']
            )
        else:
            decision_boundary_plot = None

        # Формуємо список графіків для виведення
        output_plots = [learning_curve_plot, metrics_plot, cm_plot, misclassifications_plot]

        if roc_plot is not None:
            output_plots.append(roc_plot)

        if decision_boundary_plot is not None:
            output_plots.append(decision_boundary_plot)

        # Повертаємо результати
        return [output_text] + output_plots + [None] * (6 - len(output_plots))

    except Exception as e:
        error_message = f"Помилка при навчанні моделей: {str(e)}"
        # Повертаємо помилку та порожні графіки
        return [error_message] + [None] * 6

In [22]:
# Створення Gradio інтерфейсу
with gr.Blocks(title="Нейронні мережі для задач класифікації") as demo:
    gr.Markdown("# Нейронні мережі для задач класифікації")
    gr.Markdown("У цьому інтерфейсі ви можете експериментувати з різними архітектурами нейронних мереж та порівнювати їх з класичними методами машинного навчання")

    with gr.Row():
        with gr.Column(scale=1):
            # Вибір набору даних
            variant = gr.Dropdown(
                choices=[
                    (f"Варіант {i}: {name}", i) for i, name in enumerate([
                        "Breast Cancer Wisconsin",
                        "Fashion MNIST",
                        "Pima Indians Diabetes",
                        "Wine Quality",
                        "Wine Dataset",
                        "Titanic",
                        "Heart Disease",
                        "Iris",
                        "Mushroom Dataset",
                        "MNIST Digits"
                    ], 1)
                ],
                value=1,
                label="Варіант набору даних"
            )

            test_size = gr.Slider(
                minimum=10,
                maximum=50,
                value=20,
                step=5,
                label="Розмір тестової вибірки (%)"
            )

            random_state = gr.Slider(
                minimum=0,
                maximum=100,
                value=42,
                step=1,
                label="Random State"
            )

            with gr.Tab("Архітектура моделей"):
                keras_architecture = gr.Dropdown(
                    choices=[
                        ("Проста", "simple"),
                        ("Глибока", "deep"),
                        ("Широка", "wider"),
                        ("Згорткова (тільки для MNIST/Fashion)", "cnn")
                    ],
                    value="simple",
                    label="Архітектура Keras моделі"
                )

                sklearn_model_type = gr.Dropdown(
                    choices=[
                        ("Багатошаровий перцептрон", "mlp"),
                        ("Логістична регресія", "logistic")
                    ],
                    value="mlp",
                    label="Тип Scikit-learn моделі"
                )

                with gr.Row():
                    first_layer_units = gr.Slider(
                        minimum=4,
                        maximum=128,
                        value=32,
                        step=4,
                        label="Нейрони в першому шарі"
                    )

                    second_layer_units = gr.Slider(
                        minimum=2,
                        maximum=64,
                        value=16,
                        step=2,
                        label="Нейрони в другому шарі"
                    )

                activation = gr.Dropdown(
                    choices=[
                        ("ReLU", "relu"),
                        ("Sigmoid", "sigmoid"),
                        ("Tanh", "tanh")
                    ],
                    value="relu",
                    label="Функція активації"
                )

                dropout_rate = gr.Slider(
                    minimum=0.0,
                    maximum=0.5,
                    value=0.2,
                    step=0.05,
                    label="Dropout Rate"
                )

            with gr.Tab("Параметри навчання"):
                with gr.Row():
                    optimizer = gr.Dropdown(
                        choices=[
                            ("Adam", "adam"),
                            ("SGD", "sgd"),
                            ("RMSprop", "rmsprop")
                        ],
                        value="adam",
                        label="Оптимізатор (Keras)"
                    )

                    learning_rate = gr.Slider(
                        minimum=0.0001,
                        maximum=0.01,
                        value=0.001,
                        step=0.0001,
                        label="Швидкість навчання"
                    )

                with gr.Row():
                    batch_size = gr.Dropdown(
                        choices=[8, 16, 32, 64, 128],
                        value=32,
                        label="Розмір батчу (Keras)"
                    )

                    epochs = gr.Slider(
                        minimum=10,
                        maximum=200,
                        value=50,
                        step=10,
                        label="Максимальна кількість епох (Keras)"
                    )

                with gr.Row():
                    alpha = gr.Slider(
                        minimum=0.0001,
                        maximum=0.1,
                        value=0.0001,
                        step=0.0001,
                        label="Alpha (регуляризація)"
                    )

                    max_iter = gr.Slider(
                        minimum=100,
                        maximum=1000,
                        value=200,
                        step=50,
                        label="Максимальна кількість ітерацій (Scikit-learn)"
                    )

            submit_btn = gr.Button("Навчити та оцінити моделі")

        with gr.Column(scale=2):
            output_text = gr.Markdown(label="Результати")

            with gr.Tab("Навчання"):
                learning_curve_plot = gr.Plot(label="Криві навчання (Keras)")

            with gr.Tab("Метрики"):
                metrics_plot = gr.Plot(label="Порівняння метрик моделей")

            with gr.Tab("Матриці помилок"):
                cm_plot = gr.Plot(label="Матриці помилок")

            with gr.Tab("Неправильно класифіковані зразки"):
                misclassifications_plot = gr.Plot(label="Приклади помилкової класифікації")

            with gr.Tab("ROC криві"):
                roc_plot = gr.Plot(label="ROC криві (тільки для бінарної класифікації)")

            with gr.Tab("Межі прийняття рішень"):
                decision_boundary_plot = gr.Plot(label="Межі прийняття рішень (тільки для 2D даних)")

    # Підключення функції до кнопки
    submit_btn.click(
        fn=gradio_interface,
        inputs=[variant, test_size, keras_architecture, sklearn_model_type,
               first_layer_units, second_layer_units, activation, learning_rate,
               optimizer, batch_size, epochs, dropout_rate, alpha, max_iter, random_state],
        outputs=[output_text, learning_curve_plot, metrics_plot, cm_plot,
                misclassifications_plot, roc_plot, decision_boundary_plot]
    )

# Запуск інтерфейсу
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6261d2cf9fb6acd6fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


 ---
###Завдання 7.  Документування та висновки
- Опишіть методологію і отримані результати
- Порівняйте різні підходи та зробіть висновки
- Запропонуйте можливі покращення